In [2]:
import cvxpy as cp
import numpy as np
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product

## Questão 1: Arranha-céus!

### a)

### b)

### c)

\begin{align*}
    \text{Encontrar }   & x_{ijk} \hspace{20pt} \text{(Alocar o número } k \text{ à posição } (i,j) \text{ do tabuleiro)}\\
    \text{sujeito a }   & \sum_{k} x_{ijk} = 1 \quad \forall  i,j  \hspace{20pt} \text{(Alocar apenas um número em cada casa)} \\
                        & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Apenas um número em cada coluna)} \\
                        & \sum_{j} x_{ijk} = 1 \quad \forall  i,k  \hspace{20pt} \text{(Apenas um número em cada linha)} \\
                        & x_{ijk} = 1 \quad \forall i,j,k: \: \text{board}(i,j) = k \hspace{20pt} \text{(Fixar valores do tabuleiro inicial)}\\
                        & x_{ijk}\in \mathbb{B} \quad \forall  i,j,k  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
\end{align*}

## Questão 2: Plantação de tomates!

### a) Estratégia gulosa
A estratégia gulosa funciona para o exemplo simples onde o jardim é dado por

| 5 | 3 | 5 |

Um contra exemplo à essa estratégia é o jardim 

| 3 | 5 | 3 |

Nesse último caso, a estratégia gulosa nos dá uma delicioncisade total de 5, sendo que o ótimo é na realidade 6.

### b)

### c) Programa linear

\begin{align*}
    \text{Maximizar }   & \sum_{i=1}^{n} T_i x_i \hspace{20pt} \text{(Deliciosidade total)}\\
    \text{sujeito a }   & \; x_i + x_{i+1} = 1 \quad \forall i \in \{1,..., n-1\}  \hspace{20pt} \text{(Não plantar dois tomates em casas adjacentes)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - plantar ou não plantar tomate em cada casa)} \\
                        & T_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Deliciosidades inteiras e positivas)} \\
\end{align*}

### d)



## Questão 3: Dois problemas fundamentais em grafos bipartidos

### a)

Dado um grafo $G(V, E)$, observamos primeiramente que um emparelhamento qualquer $Y$ pode envolver, no melhor dos casos, todos os seus nós. Isso implica que  $|Y| <= |V|/2$ já que cada aresta de $Y$ não pode compartilhar nós com outras, e cada aresta cobre exatamente 2 nós. Já a cobertura 

### b) Emparelhamento máximo

\begin{align*}
    \text{Maximizar }   & \sum_{(i,j)\in E} x_{ij} \hspace{20pt} \text{(Maximizar conjunto de emparalhamento)}\\
    \text{sujeito a }   & \sum_{j} x_{ij} = 1 \quad \forall i \in V  \hspace{20pt} \text{(Apenas uma aresta chegando em um vértice }i\text{)} \\
                        & \sum_{i} x_{ij} = 1 \quad \forall j \in V  \hspace{20pt} \text{(Apenas uma aresta saindo em um vértice }j\text{)} \\
                        & x_{ij}\in \mathbb{B} \quad \forall  (i,j)\in E  \hspace{20pt} \text{(VAriáveis de decisão binárias)} \\
                                                & \\
    \text{onde }        & G(V,E) \text{ é um grafo com conjunto de nós } V \text{e conjunto de arestas } E     
\end{align*}

### c) Cobertura mínima

\begin{align*}
    \text{Minimizar }   & \sum_{i\in V} x_{i} \hspace{20pt} \text{(Minimizar conjunto de cobertura)}\\
    \text{sujeito a }   & \; x_{i} + x_{j} >= 1 \quad \forall (i,j) \in E  \hspace{20pt} \text{(Toda aresta possui um de seus nós na cobertura} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i\in V  \hspace{20pt} \text{(Variáveis de decisão binárias)} \\
                        & \\
    \text{onde }        & G(V,E) \text{ é um grafo com conjunto de nós } V \text{e conjunto de arestas } E             
\end{align*}

### d)



## Questão 4: O Princípio da Casa dos Pombos

### a)

### b)

### c)



## Questão 5: Pudim, o pinguim comilão.

### a)

O problema só é factível se não houver algum segmento da viagem de comprimento maior de que $m$, a distância máxima que Pudim pode viajar sem comer. Mais formalmente, o problema é factível somente se $F_i - F_{i-1} \leq m \quad \forall i \in \{ 2,...,n \}$.

### b)

\begin{align*}
    \text{Minimizar }   & \sum_{i=1}^{n} x_i \hspace{20pt} \text{(Total de paradas realizadas)}\\
    \text{sujeito a }   & \sum_{i\in Cj}x_i \leq  0 \quad \forall j \in \{1,..., n\}  \hspace{20pt} \text{(Toda parada possui uma parada anterior dentro do alcance máximo)} \\
                        & x_{i}\in \mathbb{B} \quad \forall  i  \hspace{20pt} \text{(Variáveis de decisão binárias - parar ou não no buraco de pesca)} \\
                        & F_{i}\in \mathbb{Z} \quad \forall  i  \hspace{20pt} \text{(Distâncias inteiras e positivas)} \\
                        & \\
    \text{onde }   & Cj \equiv \{ i:\, F_j - F_i <= m \} \hspace{20pt} \text{(Conjunto de paradas anteriores a } j \text{ com distância menor que } m\text{)}
\end{align*}

### c)

Seja $S(i)$ a solução ótima para o percurso até a parada $i$, sendo então o conjunto de paradas restantes do trajeto $R = \{ i+1,..,n \}$. Provaremos por absurdo que a escolha gulosa é a melhor escolha para o próximo passo da indução. Se a escolha gulosa é dada pela parada $k\in R$, mas ela não minimiza $|S(k)|$, então temos duas alternativas:

1) Existe algum $k'>k$ ao alcance de Pudim quando este está em $i$. Nesse caso, temos um absurdo pois a escolha gulosa implica em selecionar a última parada factível à partir de $i$.
2) A suposta solução melhor que a gulosa envolve algum $k' < k$. Como $k$ está ao alcance de Pudim em $i$, então podemos pular $k'$ e continuar com o mesmo número de paradas. 

Logo, se $S(i)$ é a solução ótima para $\{1,...,i\}$, então $S(k)$ dada pela estratégia gulosa é a solução ótima para $\{1,...,i,...,k\}$. O caso base da indução é dado por $S(1) = \{0\}$, o ponto de início da viagem.  

[talvez seja necessário usar indução completa sobre todas as paradas ao alcance de pudim em $i$, ao invés do raciocínio acima] 

In [51]:
# problem input
F = [0, 3, 4, 6, 10, 12]
m = 4

# decision variables
decisionVars = cp.Variable(len(F), boolean=True, name="stop")

# constraints
constraints = []

# creating list of valid previous stops for each stop in the journey
allValidStops = []
for i in range(1, len(F)):
    validStops = []
    for j in range(i-1, -1, -1):
        if F[i] - F[j] <= m:
            validStops += [j]
    allValidStops.append(validStops)

# all stops must have at least one previous valid stop
for i in range(0, len(F)-1):
    constraints += [cp.sum(decisionVars[allValidStops[i]]) >= 1]

# start and end of the journey
constraints += [decisionVars[-1] == 1]
constraints += [decisionVars[0] == 1]

objective = cp.Minimize(cp.sum(decisionVars))

# solving 
lp = cp.Problem(objective, constraints)
lp.solve(solver="MOSEK", verbose=False)

# printing model and solution
print(lp)
print()
print("Solução: ", decisionVars.value)

minimize Sum(stop, None, False)
subject to 1.0 <= Sum(stop[0], None, False)
           1.0 <= Sum(stop[1, 0], None, False)
           1.0 <= Sum(stop[2, 1], None, False)
           1.0 <= Sum(stop[3], None, False)
           1.0 <= Sum(stop[4], None, False)
           stop[5] == 1.0
           stop[0] == 1.0

Solução:  [1. 0. 1. 1. 1. 1.]
